In [ ]:
!pip install OpenAI
!pip install langchain
!pip install langchain_community
!pip install Cohere
!pip install langchain-openai langchain-cohere python-dotenv


In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
from google.colab import userdata

In [ ]:
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key

In [ ]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI


In [ ]:



CSV_PATH = "https://raw.githubusercontent.com/giridhar276/genai/refs/heads/main/datasets/Bank_Customer_conversations.csv"
TEXT_COL = "customer_text"
MODEL = "gpt-4o-mini"
TEMPERATURE = 0
TIMEOUT = 60

OUTPUT_PATH = CSV_PATH.replace(".csv", "_with_summary.csv")

assert os.getenv("OPENAI_API_KEY"), "Please set OPENAI_API_KEY in your environment."


In [ ]:

PROMPT = (
    "Summarize the customer's message in ONE clear sentence focusing on intent/issue and requested action. "
    "Do not add details that are not present. "
    "Return only the summary text on a single line.\n\n"
    "CUSTOMER TEXT:\n"
    '\"\"\"{text}\"\"\"\n'
    "Summary:"
)


In [ ]:

df = pd.read_csv(CSV_PATH)
df = df[[TEXT_COL]].copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()

print("Rows:", len(df))
df.head(3)


In [ ]:

llm = ChatOpenAI(model=MODEL, temperature=TEMPERATURE, timeout=TIMEOUT)

def summarize_one(text) :
    return llm.invoke(PROMPT.format(text=text)).content.strip()


In [ ]:

summaries = []
for txt in df[TEXT_COL].tolist():
    summaries.append(summarize_one(txt))

df["summary"] = summaries


In [ ]:

df.to_csv("summary.csv", index=False)
print(f"Saved: {OUTPUT_PATH}")
df[[TEXT_COL, "summary"]].head(10)
